# Tutoriel : Analyse de donnees Eye-Tracking

Ce notebook vous guide pas a pas dans l'analyse de donnees eye-tracking de l'etude SDS2.

## Objectifs

A la fin de ce tutoriel, vous saurez :
1. Charger et explorer les donnees brutes d'un eye-tracker Tobii
2. Nettoyer et pre-traiter les donnees
3. Calculer les metriques cles (fixations, saccades, pupilles)
4. Creer des visualisations (cartes de chaleur, trajectoires)
5. Comparer les groupes Patient vs Controle
6. Integrer les donnees comportementales BORIS

## Prerequis

- Connaissance de base de Python (pandas, matplotlib)
- Le projet installe (`pip install -e ".[dev]"`)
- Les donnees dans le dossier `Data/`

---

## Partie 1 : Configuration et verification

Commencons par importer les bibliotheques necessaires et verifier que tout est bien installe.

In [1]:
# Imports standards Python
from pathlib import Path
import warnings

# Bibliotheques de manipulation de donnees
import pandas as pd
import numpy as np

# Visualisation
import matplotlib.pyplot as plt

# Desactiver les warnings pour une lecture plus claire
warnings.filterwarnings('ignore')

# Configuration matplotlib pour de beaux graphiques
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("Imports reussis !")

Imports reussis !


In [2]:
# Imports du pipeline tobii
# Ces modules sont le coeur du projet d'analyse

from tobii_pipeline import load_recording  # Charger un fichier TSV
from tobii_pipeline.cleaner import clean_recording, filter_eye_tracker  # Nettoyer les donnees
from tobii_pipeline.parser import parse_filename  # Extraire les metadonnees du nom de fichier

# Fonctions d'analyse
from tobii_pipeline.analysis.metrics import (
    compute_recording_summary,  # Calculer toutes les metriques
    compute_validity_rate,      # Qualite des donnees
    compute_gaze_dispersion,    # Dispersion du regard
    compute_pupil_variability,  # Variabilite pupillaire
)

# Fonctions de visualisation
from tobii_pipeline.analysis.plots import (
    plot_recording_summary,  # Vue d'ensemble
    plot_gaze_scatter,       # Points de regard
    plot_pupil_timeseries,   # Evolution pupillaire
)

print("Pipeline tobii importe avec succes !")

ModuleNotFoundError: No module named 'tobii_pipeline'

In [ ]:
# Verification de la presence des donnees
# IMPORTANT : Vous devez avoir le dossier Data/ a la racine du projet

# Chemins vers les donnees
DATA_DIR = Path("../Data")
TOBII_DIRS = [DATA_DIR / "data_G" / "Tobii", DATA_DIR / "data_L" / "Tobii"]
BORIS_DIRS = [DATA_DIR / "data_G" / "Boris", DATA_DIR / "data_L" / "Boris"]

# Verifier que les dossiers existent
if not DATA_DIR.exists():
    print("ATTENTION : Le dossier Data/ n'existe pas !")
    print("Veuillez placer vos donnees dans le dossier Data/ a la racine du projet.")
    print("Structure attendue :")
    print("  Data/")
    print("    data_G/")
    print("      Tobii/  <- fichiers .tsv")
    print("      Boris/  <- fichiers .xlsx")
    print("    data_L/")
    print("      Tobii/")
    print("      Boris/")
else:
    # Compter les fichiers disponibles
    tobii_files = []
    for d in TOBII_DIRS:
        if d.exists():
            tobii_files.extend(list(d.glob("*.tsv")))
    
    print(f"Donnees trouvees : {len(tobii_files)} fichiers Tobii")
    
    if tobii_files:
        print("\nExemples de fichiers :")
        for f in tobii_files[:3]:
            print(f"  - {f.name}")

---

## Partie 2 : Chargement d'un enregistrement

Un enregistrement = une session d'eye-tracking d'un participant a une visite donnee.

Le fichier TSV contient toutes les mesures capturees par le Tobii a 100 Hz (100 mesures par seconde).

In [ ]:
# Selectionner un fichier pour l'analyse
# Nous allons utiliser le premier fichier disponible

if not tobii_files:
    raise FileNotFoundError("Aucun fichier Tobii trouve. Ajoutez vos donnees dans Data/")

# Prendre le premier fichier comme exemple
fichier_exemple = tobii_files[0]
print(f"Fichier selectionne : {fichier_exemple.name}")

# Extraire les metadonnees du nom de fichier
# Le nom suit la convention : ID_Participant_Etude_Groupe_Mois_Visite_Date
metadata = parse_filename(fichier_exemple.name)

print("\nMetadonnees extraites :")
for cle, valeur in metadata.items():
    print(f"  {cle}: {valeur}")

In [ ]:
# Charger les donnees brutes
# La fonction load_recording() lit le fichier TSV et le convertit en DataFrame pandas

print("Chargement en cours...")
df_brut = load_recording(fichier_exemple)

print(f"\nDonnees chargees : {len(df_brut):,} lignes x {len(df_brut.columns)} colonnes")
print(f"Duree approximative : {len(df_brut) / 100:.1f} secondes (a 100 Hz)")

In [ ]:
# Explorer la structure des donnees
# Chaque ligne = une mesure (1/100e de seconde)
# Chaque colonne = un type de mesure

print("Apercu des premieres lignes :")
df_brut.head()

In [ ]:
# Liste des colonnes disponibles
# Le Tobii capture beaucoup d'informations differentes

print(f"Colonnes disponibles ({len(df_brut.columns)}) :\n")

# Grouper les colonnes par categorie pour plus de clarte
colonnes_importantes = {
    "Temps": ["Recording timestamp"],
    "Position du regard": ["Gaze point X", "Gaze point Y"],
    "Pupilles": ["Pupil diameter left", "Pupil diameter right"],
    "Validite": ["Validity left", "Validity right"],
    "Capteur": ["Sensor"],
}

for categorie, colonnes in colonnes_importantes.items():
    print(f"{categorie}:")
    for col in colonnes:
        if col in df_brut.columns:
            print(f"  - {col}")

### Comprendre les colonnes cles

| Colonne | Description | Unite |
|---------|-------------|-------|
| `Recording timestamp` | Temps depuis le debut | Microsecondes |
| `Gaze point X`, `Gaze point Y` | Position du regard sur l'ecran | Pixels (0-1920, 0-1080) |
| `Pupil diameter left/right` | Taille des pupilles | Millimetres |
| `Validity left/right` | Qualite de la mesure | "Valid" ou "Invalid" |
| `Sensor` | Type de capteur | "Eye Tracker", "Accelerometer", etc. |

---

## Partie 3 : Nettoyage des donnees

Les donnees brutes necessitent un nettoyage avant analyse :
1. **Conversion des decimales** : Le Tobii exporte avec des virgules (format europeen)
2. **Filtrage** : Garder uniquement les donnees de l'eye-tracker (pas l'accelerometre)
3. **Validation** : Identifier les mesures invalides

In [ ]:
# Etape 1 : Nettoyage de base
# La fonction clean_recording() gere automatiquement :
# - La conversion des virgules en points (3,14 -> 3.14)
# - La conversion des types de donnees

df_propre = clean_recording(df_brut)

# Verifier que les colonnes numeriques sont bien converties
print("Types de donnees apres nettoyage :")
print(df_propre[["Gaze point X", "Pupil diameter left"]].dtypes)

In [ ]:
# Etape 2 : Filtrer pour ne garder que les donnees eye-tracker
# Le fichier contient aussi des donnees d'accelerometre et gyroscope
# que nous n'utilisons pas pour l'analyse du regard

print("Types de capteurs dans les donnees :")
print(df_propre["Sensor"].value_counts())

# Filtrer
df = filter_eye_tracker(df_propre)

print(f"\nApres filtrage : {len(df):,} lignes (vs {len(df_propre):,} avant)")

In [ ]:
# Etape 3 : Evaluer la qualite des donnees
# La colonne "Validity" indique si le Tobii a reussi a detecter l'oeil

# Calculer le taux de validite (% de mesures exploitables)
taux_validite = compute_validity_rate(df)

print(f"Taux de validite : {taux_validite:.1%}")

if taux_validite > 0.8:
    print("Bonne qualite de donnees")
elif taux_validite > 0.5:
    print("Qualite acceptable, mais des donnees sont manquantes")
else:
    print("Attention : beaucoup de donnees manquantes")

In [ ]:
# Visualiser les donnees manquantes au cours du temps
# Cela permet de voir si les pertes de signal sont concentrees ou dispersees

fig, ax = plt.subplots(figsize=(14, 4))

# Creer un indicateur de validite (1 = valide, 0 = invalide)
validite = ((df["Validity left"] == "Valid") & (df["Validity right"] == "Valid")).astype(int)

# Calculer la moyenne mobile sur 1 seconde (100 echantillons)
validite_lissee = validite.rolling(window=100, min_periods=1).mean()

# Convertir le temps en secondes
temps_secondes = (df["Recording timestamp"] - df["Recording timestamp"].iloc[0]) / 1_000_000

ax.fill_between(temps_secondes, validite_lissee, alpha=0.7, color='steelblue')
ax.set_xlabel("Temps (secondes)")
ax.set_ylabel("Taux de validite")
ax.set_title("Qualite du signal au cours de l'enregistrement")
ax.set_ylim(0, 1.05)
ax.axhline(y=0.8, color='green', linestyle='--', alpha=0.5, label='Seuil 80%')
ax.legend()

plt.tight_layout()
plt.show()

---

## Partie 4 : Calcul des metriques

Maintenant que les donnees sont propres, calculons les metriques cles qui caracterisent le comportement visuel.

In [ ]:
# Calculer toutes les metriques d'un coup
# La fonction compute_recording_summary() fait tout le travail :
# - Detection des fixations et saccades (via pymovements)
# - Statistiques pupillaires
# - Metriques de qualite

print("Calcul des metriques en cours...")
print("(Cela peut prendre quelques secondes pour la detection des evenements)")

metriques = compute_recording_summary(df)

print("\nCalcul termine !")

In [ ]:
# Afficher les metriques de facon structuree
# Les metriques sont organisees en categories

print("=" * 60)
print("RESUME DES METRIQUES")
print("=" * 60)

# Qualite des donnees
print("\n--- QUALITE ---")
qualite = metriques.get("quality", {})
print(f"  Taux de validite (2 yeux) : {qualite.get('validity_rate', 0):.1%}")
print(f"  Taux de validite (1 oeil) : {qualite.get('validity_rate_either', 0):.1%}")

# Regard
print("\n--- REGARD ---")
regard = metriques.get("gaze", {})
centre = regard.get("center", (np.nan, np.nan))
print(f"  Centre moyen : ({centre[0]:.0f}, {centre[1]:.0f}) pixels")
print(f"  Dispersion : {regard.get('dispersion', np.nan):.1f} pixels")

# Pupilles
print("\n--- PUPILLES ---")
pupille = metriques.get("pupil", {})
stats_pupille = pupille.get("stats", {})
print(f"  Diametre moyen : {stats_pupille.get('mean', np.nan):.2f} mm")
print(f"  Variabilite (CV) : {pupille.get('variability', np.nan):.3f}")

# Fixations
print("\n--- FIXATIONS ---")
fixation = metriques.get("fixation", {})
print(f"  Nombre : {fixation.get('count', 0)}")
print(f"  Duree moyenne : {fixation.get('duration_mean_ms', np.nan):.0f} ms")

# Saccades
print("\n--- SACCADES ---")
saccade = metriques.get("saccade", {})
print(f"  Nombre : {saccade.get('count', 0)}")
print(f"  Amplitude moyenne : {saccade.get('amplitude_mean_deg', np.nan):.1f} degres")

### Interpretation des metriques

- **Dispersion elevee** (> 300 px) : Le regard explore largement l'ecran
- **Fixations longues** (> 400 ms) : Traitement visuel approfondi ou difficulte
- **Variabilite pupillaire elevee** (> 0.15) : Charge cognitive fluctuante
- **Beaucoup de saccades** : Exploration active, recherche visuelle

---

## Partie 5 : Visualisations

Les graphiques permettent de comprendre intuitivement le comportement visuel.

In [ ]:
# Vue d'ensemble complete de l'enregistrement
# Cette fonction genere plusieurs graphiques en un seul appel

fig = plot_recording_summary(df, figsize=(16, 12))

# Ajouter un titre avec les informations du participant
titre = f"Participant: {metadata.get('participant', '?')} | "
titre += f"Groupe: {metadata.get('group', '?')} | "
titre += f"Visite: M{metadata.get('month', '?')}"
fig.suptitle(titre, fontsize=14, y=1.02)

plt.tight_layout()
plt.show()

In [ ]:
# Carte de chaleur du regard (heatmap)
# Montre ou le participant a regarde le plus souvent
# Les zones chaudes (jaune/rouge) = plus regardees

from tobii_pipeline.adapters.mne_adapter import create_gaze_heatmap_mne

fig, ax = plt.subplots(figsize=(12, 8))

# Extraire les positions de regard valides
masque_valide = (df["Validity left"] == "Valid") | (df["Validity right"] == "Valid")
x = df.loc[masque_valide, "Gaze point X"].dropna()
y = df.loc[masque_valide, "Gaze point Y"].dropna()

# Creer la heatmap
create_gaze_heatmap_mne(x.values, y.values, ax=ax, screen_size=(1920, 1080))

ax.set_title("Carte de chaleur du regard")
ax.set_xlabel("Position X (pixels)")
ax.set_ylabel("Position Y (pixels)")

plt.tight_layout()
plt.show()

In [ ]:
# Evolution du diametre pupillaire au cours du temps
# La pupille se dilate avec l'effort mental et se contracte avec la lumiere

fig, ax = plt.subplots(figsize=(14, 5))

# Convertir le temps en secondes
temps = (df["Recording timestamp"] - df["Recording timestamp"].iloc[0]) / 1_000_000

# Tracer les deux yeux
ax.plot(temps, df["Pupil diameter left"], alpha=0.5, label="Oeil gauche", linewidth=0.5)
ax.plot(temps, df["Pupil diameter right"], alpha=0.5, label="Oeil droit", linewidth=0.5)

# Ajouter une moyenne mobile pour voir la tendance
moyenne_gauche = df["Pupil diameter left"].rolling(window=500, min_periods=1).mean()
ax.plot(temps, moyenne_gauche, color='blue', linewidth=2, label="Tendance (gauche)")

ax.set_xlabel("Temps (secondes)")
ax.set_ylabel("Diametre pupillaire (mm)")
ax.set_title("Evolution du diametre pupillaire")
ax.legend()
ax.set_ylim(2, 8)  # Plage physiologique normale

plt.tight_layout()
plt.show()

---

## Partie 6 : Comparaison Patient vs Controle

L'objectif principal de l'etude est de comparer les comportements visuels entre les deux groupes.
Pour cela, nous devons charger plusieurs enregistrements et calculer leurs metriques.

In [ ]:
# Charger et analyser plusieurs enregistrements
# ATTENTION : Cette cellule peut prendre plusieurs minutes

from tqdm.notebook import tqdm  # Barre de progression

# Limiter le nombre de fichiers pour l'exemple (ajustez selon vos besoins)
MAX_FICHIERS = 10  # Mettre None pour tout charger

fichiers_a_analyser = tobii_files[:MAX_FICHIERS] if MAX_FICHIERS else tobii_files
print(f"Analyse de {len(fichiers_a_analyser)} enregistrements...")

resultats = []

for fichier in tqdm(fichiers_a_analyser, desc="Analyse"):
    try:
        # Charger et nettoyer
        df_temp = load_recording(fichier, nrows=50000)  # Limiter les lignes pour accelerer
        df_temp = clean_recording(df_temp)
        df_temp = filter_eye_tracker(df_temp)
        
        # Extraire les metadonnees
        meta = parse_filename(fichier.name)
        
        # Calculer les metriques
        metriques_temp = compute_recording_summary(df_temp)
        
        # Stocker les resultats
        resultats.append({
            "fichier": fichier.name,
            "participant": meta.get("participant", ""),
            "groupe": "Patient" if meta.get("group") == "P" else "Controle",
            "mois": meta.get("month", 0),
            "validity_rate": metriques_temp.get("quality", {}).get("validity_rate", np.nan),
            "gaze_dispersion": metriques_temp.get("gaze", {}).get("dispersion", np.nan),
            "pupil_variability": metriques_temp.get("pupil", {}).get("variability", np.nan),
            "fixation_count": metriques_temp.get("fixation", {}).get("count", 0),
            "fixation_duration": metriques_temp.get("fixation", {}).get("duration_mean_ms", np.nan),
        })
    except Exception as e:
        print(f"Erreur avec {fichier.name}: {e}")
        continue

# Convertir en DataFrame
df_resultats = pd.DataFrame(resultats)
print(f"\n{len(df_resultats)} enregistrements analyses avec succes")

In [ ]:
# Afficher un apercu des resultats
df_resultats.head(10)

In [ ]:
# Comparer les groupes
# Calculer les moyennes par groupe

if len(df_resultats) > 0 and "groupe" in df_resultats.columns:
    comparaison = df_resultats.groupby("groupe").agg({
        "validity_rate": ["mean", "std", "count"],
        "gaze_dispersion": ["mean", "std"],
        "pupil_variability": ["mean", "std"],
        "fixation_duration": ["mean", "std"],
    }).round(3)
    
    print("\nComparaison Patient vs Controle :")
    print(comparaison)
else:
    print("Pas assez de donnees pour la comparaison")

In [ ]:
# Visualisation : Comparaison des distributions

if len(df_resultats) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    metriques_a_comparer = [
        ("gaze_dispersion", "Dispersion du regard (px)"),
        ("pupil_variability", "Variabilite pupillaire (CV)"),
        ("fixation_duration", "Duree des fixations (ms)"),
        ("fixation_count", "Nombre de fixations"),
    ]
    
    couleurs = {"Patient": "#e74c3c", "Controle": "#3498db"}
    
    for ax, (col, titre) in zip(axes.flatten(), metriques_a_comparer):
        for groupe in ["Patient", "Controle"]:
            donnees = df_resultats[df_resultats["groupe"] == groupe][col].dropna()
            if len(donnees) > 0:
                ax.hist(donnees, alpha=0.6, label=groupe, color=couleurs[groupe], bins=10)
        
        ax.set_xlabel(titre)
        ax.set_ylabel("Frequence")
        ax.legend()
        ax.set_title(f"Distribution : {titre}")
    
    plt.tight_layout()
    plt.show()
else:
    print("Pas assez de donnees pour la visualisation")

---

## Partie 7 : Analyse longitudinale

L'etude suit les participants sur 36 mois. Observons comment les metriques evoluent dans le temps.

In [ ]:
# Evolution des metriques au cours du temps

if len(df_resultats) > 0 and "mois" in df_resultats.columns:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    for ax, (col, titre) in zip(axes.flatten(), metriques_a_comparer):
        for groupe in ["Patient", "Controle"]:
            donnees_groupe = df_resultats[df_resultats["groupe"] == groupe]
            
            # Calculer la moyenne par mois
            moyennes = donnees_groupe.groupby("mois")[col].mean()
            
            if len(moyennes) > 0:
                ax.plot(moyennes.index, moyennes.values, 
                       'o-', label=groupe, color=couleurs[groupe], 
                       linewidth=2, markersize=8)
        
        ax.set_xlabel("Mois")
        ax.set_ylabel(titre)
        ax.set_title(f"Evolution : {titre}")
        ax.legend()
        ax.set_xticks([0, 12, 24, 36])
        ax.set_xticklabels(["M0", "M12", "M24", "M36"])
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("Pas assez de donnees pour l'analyse longitudinale")

---

## Partie 8 : Integration des donnees BORIS (optionnel)

Les donnees BORIS contiennent le codage comportemental realise par les observateurs.
Cela permet de croiser les metriques eye-tracking avec les comportements observes.

In [ ]:
# Verifier la disponibilite des donnees BORIS

boris_files = []
for d in BORIS_DIRS:
    if d.exists():
        boris_files.extend(list(d.glob("*_agregated.xlsx")))

print(f"Fichiers BORIS trouves : {len(boris_files)}")

if boris_files:
    print("\nExemples :")
    for f in boris_files[:3]:
        print(f"  - {f.name}")

In [ ]:
# Charger un fichier BORIS

if boris_files:
    from boris_pipeline import load_boris_file
    
    boris_exemple = boris_files[0]
    print(f"Chargement de : {boris_exemple.name}")
    
    df_boris = load_boris_file(boris_exemple, file_type="aggregated")
    
    print(f"\n{len(df_boris)} evenements comportementaux")
    print("\nColonnes :")
    print(df_boris.columns.tolist())
    
    print("\nApercu :")
    display(df_boris.head())
else:
    print("Pas de fichiers BORIS disponibles")

In [ ]:
# Explorer les comportements codes

if boris_files and 'df_boris' in dir():
    print("Comportements observes :")
    print(df_boris["Behavior"].value_counts())
else:
    print("Donnees BORIS non disponibles")

In [ ]:
# Exemple d'analyse croisee : diametre pupillaire par comportement

if boris_files and 'df_boris' in dir():
    from integration.cross_modal import compute_pupil_per_behavior
    
    # Utiliser le meme participant pour les deux sources de donnees
    # (le fichier Tobii charge au debut du notebook)
    
    try:
        pupil_par_comportement = compute_pupil_per_behavior(df, df_boris)
        
        print("Diametre pupillaire moyen par comportement :")
        for comportement, stats in pupil_par_comportement.items():
            if isinstance(stats, dict) and 'mean' in stats:
                print(f"  {comportement}: {stats['mean']:.2f} mm")
    except Exception as e:
        print(f"Erreur lors de l'analyse croisee : {e}")
        print("(Les fichiers Tobii et BORIS doivent correspondre au meme participant)")
else:
    print("Donnees BORIS non disponibles")

---

## Partie 9 : Conclusion et prochaines etapes

### Ce que nous avons appris

1. **Chargement** : Les fichiers Tobii TSV contiennent des mesures a 100 Hz
2. **Nettoyage** : Conversion des decimales et filtrage necessaires
3. **Metriques** : Fixations, saccades, pupilles caracterisent le comportement visuel
4. **Visualisation** : Heatmaps et timeseries revelent les patterns
5. **Comparaison** : Tests statistiques pour detecter les differences entre groupes
6. **Integration** : BORIS permet de lier eye-tracking et comportement

### Pour aller plus loin

- **Scripts automatises** : Utilisez `run_analysis.py` pour des analyses completes
- **Figures de publication** : Le module `pub_plots.py` genere des graphiques professionnels
- **Tests statistiques** : Le module `stats.py` contient t-tests, Mann-Whitney, etc.
- **Preprocessing avance** : `postprocess.py` gere interpolation, clignements, outliers

### Ressources

- Documentation technique : `CLAUDE.md` a la racine du projet
- README detaille : `README.md` avec explications en francais
- Tests : `pytest` pour verifier l'installation

In [ ]:
print("Tutoriel termine ! Bonne analyse.")